# Linear Regression from Scratch
Goal: implement linear regression from scratch with NumPy.
Steps: derive gradients, implement training loop, plot loss.

# Linear Regression (from scratch)
Following Andrew Ng ML Course - Week 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

# Generate a synthetic linear dataset
X = 2 * np.random.rand(100, 1)     # 100 points between 0 and 2
y = 4 + 3 * X + np.random.randn(100, 1)  # y = 4 + 3x + Gaussian noise

# Plot the data
plt.scatter(X, y, alpha=0.7)
plt.xlabel("x")
plt.ylabel("y")
plt.title("Synthetic Linear Dataset")
plt.show()

